# Deep Agents Workshop - Steps 10-12

This notebook demonstrates the transition from LangChain v1 core agents to **Deep Agents**.

## What are Deep Agents?

Deep Agents are an "agent harness" built on top of LangGraph that come with built-in capabilities for complex, multi-step tasks:

- 📋 **Planning**: `write_todos` tool for task decomposition
- 📁 **File System**: `ls`, `read_file`, `write_file`, `edit_file` for context management
- 🤖 **Subagents**: `task` tool for delegating work to specialized agents
- 🧠 **Context Management**: Automatic eviction of large tool results and conversation summarization

Deep agents are inspired by applications like Claude Code, Deep Research, and Manus.

## Workshop Structure

- **Step 10**: Introduction to Deep Agents - Built-in capabilities
- **Step 11**: Subagents - Specialized delegation
- **Step 12**: Custom Middleware - Extending deep agents

---

## Setup

In [ ]:
# Install required packages (if needed)
# !pip install deepagents langchain langchain-openai langgraph python-dotenv

In [7]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

---

# Step 10: Introduction to Deep Agents

## Key Concepts

Instead of `create_agent`, we use `create_deep_agent` which automatically includes:

1. **TodoListMiddleware** - Provides `write_todos` tool for planning
2. **FilesystemMiddleware** - Provides file system tools for context management
3. **SubAgentMiddleware** - Provides `task` tool for delegation

## Comparison

```python
# Regular agent (Steps 1-9)
agent = create_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
)

# Deep agent (Step 10+)
agent = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
    # Automatically includes: write_todos, file system, task tool
)
```

In [8]:
from deepagents import create_deep_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from typing import List, Dict

# Mock calendar storage
_calendar_events: List[Dict] = []

@tool
def read_calendar(date: str = None) -> str:
    """Read calendar events. If date is provided, filter events for that date."""
    if date:
        filtered = [e for e in _calendar_events if e.get("date") == date]
        if not filtered:
            return f"No events found for {date}"
        return "\n".join([f"- {e['title']} on {e['date']} at {e['time']} in {e.get('location', 'N/A')}"
                         for e in filtered])
    
    if not _calendar_events:
        return "No events in calendar"
    
    return "\n".join([f"- {e['title']} on {e['date']} at {e['time']} in {e.get('location', 'N/A')}"
                     for e in _calendar_events])

@tool
def write_calendar(title: str, date: str, time: str, location: str = "") -> str:
    """Create a new calendar event."""
    for event in _calendar_events:
        if event["date"] == date and event["time"] == time:
            return f"Conflict: There's already an event '{event['title']}' scheduled for {date} at {time}"
    
    new_event = {"title": title, "date": date, "time": time, "location": location}
    _calendar_events.append(new_event)
    return f"Successfully created event '{title}' on {date} at {time} in {location}"

# Initialize model and checkpointer
model = init_chat_model("gpt-4o-mini", temperature=0)
checkpointer = MemorySaver()

# System prompt
SYSTEM_PROMPT = """You are a helpful calendar assistant with advanced planning capabilities. You can:
- Read calendar events using read_calendar
- Create new events using write_calendar

Built-in Deep Agent Capabilities:
- write_todos: Break down complex tasks into steps (automatically available)
- File system tools: ls, read_file, write_file, edit_file (automatically available)
  * Use these to store notes, drafts, or large amounts of information
  * Files are stored in agent state and persist within the conversation thread
- Task delegation: Spawn subagents for complex subtasks (automatically available)

When handling multi-step requests:
1. Use write_todos to plan your approach
2. Check the calendar first for conflicts
3. Use file system tools to save notes or drafts if needed
4. Execute the plan step by step

Be friendly and confirm when events are successfully created."""

# Create the deep agent
agent_10 = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar],
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
)

print("✅ Deep agent created with built-in capabilities!")

✅ Deep agent created with built-in capabilities!


### Test the Deep Agent

In [9]:
# Complex multi-step request
config = {"configurable": {"thread_id": "deep-agent-thread-1"}}

result = agent_10.invoke({
    "messages": [{
        "role": "user",
        "content": "Schedule three team meetings for next week: Monday at 2 PM, Wednesday at 3 PM, and Friday at 4 PM. All in the main conference room."
    }]
}, config=config)

print("Agent Response:")
print(result['messages'][-1].content)

# Check if todos were used
if "todos" in result:
    print("\n📋 Agent's Plan (from write_todos):")
    for todo in result["todos"]:
        status_emoji = "✅" if todo["status"] == "completed" else "⏳" if todo["status"] == "in_progress" else "📝"
        print(f"  {status_emoji} {todo['content']}")

Agent Response:
I've successfully scheduled the three team meetings for next week:

- **Monday, October 30** at **2 PM** in the **Main Conference Room**
- **Wednesday, November 1** at **3 PM** in the **Main Conference Room**
- **Friday, November 3** at **4 PM** in the **Main Conference Room**

If you need anything else, just let me know!


### Test Memory

In [4]:
# Test memory - the agent should remember what we just scheduled
result2 = agent_10.invoke({
    "messages": [{"role": "user", "content": "What meetings did we just schedule?"}]
}, config=config)

print("Agent Response:")
print(result2['messages'][-1].content)

Agent Response:
Here are the meetings we just scheduled:

1. **Team Meeting**
   - **Date:** Monday, October 30
   - **Time:** 2 PM
   - **Location:** Main Conference Room

2. **Team Meeting**
   - **Date:** Wednesday, November 1
   - **Time:** 3 PM
   - **Location:** Main Conference Room

3. **Team Meeting**
   - **Date:** Friday, November 3
   - **Time:** 4 PM
   - **Location:** Main Conference Room

If you need any further details or changes, feel free to ask!


---

# Step 11: Deep Agents with Subagents

## Key Concepts

Deep agents can delegate work to specialized **subagents** using the built-in `task` tool:

- **research-specialist**: Web search and event research
- **calendar-specialist**: Complex calendar operations
- **general-purpose**: Built-in subagent with all main agent's tools

## Benefits of Subagents

1. 🧹 **Context isolation** - Subagent work doesn't clutter main agent's context
2. 🎯 **Specialization** - Each subagent has specific tools and instructions
3. 📦 **Token efficiency** - Main agent receives only final results
4. 🔄 **Parallel potential** - Multiple subagents can work concurrently

## Subagent Configuration

```python
subagent = {
    "name": "research-specialist",
    "description": "When to use this subagent",
    "system_prompt": "How this subagent should behave",
    "tools": [web_search],
    "model": "gpt-4o-mini",
}
```

In [5]:
# Reset calendar for this example
_calendar_events.clear()

# Mock web search tool
@tool
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for events and information."""
    return f"""Search results for "{query}":

1. Classical Concert at Seoul Arts Center - Dec 20, 2025 at 7 PM
   Korean Symphony Orchestra performing Beethoven's 9th Symphony
   Tickets available at artscouncil.kr

2. Jazz Night at Blue Note - Dec 21, 2025 at 9 PM
   Live jazz featuring international artists
   Reservations: bluenote-seoul.com

3. K-Pop Festival - Dec 22, 2025 at 6 PM
   Outdoor festival with multiple performers
   Free entry, Han River Park"""

# Define specialized subagents
research_subagent = {
    "name": "research-specialist",
    "description": "Conducts in-depth research on events, concerts, and activities using web search. Use when you need detailed information that requires multiple searches.",
    "system_prompt": """You are an expert researcher specialized in finding events and activities.

Your job is to:
1. Break down the research question into searchable queries
2. Use web_search to gather information
3. Save detailed findings to /research_notes.txt using write_file
4. Return a concise summary (2-3 paragraphs max)

IMPORTANT: Keep your final response under 300 words to maintain clean context.
Use the file system to store detailed research, then return only the summary.""",
    "tools": [web_search],
    "model": "gpt-4o-mini",
}

calendar_specialist_subagent = {
    "name": "calendar-specialist",
    "description": "Handles complex calendar operations including conflict resolution and multi-event scheduling. Use for scheduling multiple events or resolving conflicts.",
    "system_prompt": """You are a calendar specialist who handles complex scheduling tasks.

Your job is to:
1. Use write_todos to plan your approach
2. Check calendar for conflicts using read_calendar
3. Schedule events one by one using write_calendar
4. Handle conflicts by suggesting alternative times
5. Save scheduling notes to /scheduling_log.txt if needed

Return a summary of what was scheduled and any conflicts encountered.""",
    "tools": [read_calendar, write_calendar],
    "model": "gpt-4o-mini",
}

# System prompt for the main agent
SUPERVISOR_PROMPT = """You are a supervisor calendar assistant that coordinates between specialized subagents.

Available subagents (use the 'task' tool to delegate):
1. research-specialist: For finding events, concerts, and activities via web search
2. calendar-specialist: For complex calendar operations and multi-event scheduling
3. general-purpose: A general subagent with all your tools (automatically available)

When to delegate:
- Use research-specialist when you need to search for events or gather information
- Use calendar-specialist for scheduling multiple events or handling complex calendar tasks
- Use general-purpose for other multi-step tasks that would clutter your context

Workflow for "find and schedule" requests:
1. Delegate to research-specialist to find events
2. Review the research summary
3. Delegate to calendar-specialist to handle scheduling
4. Confirm with the user

This keeps your context clean while still going deep on subtasks."""

# Create the deep agent with subagents
agent_11 = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar, web_search],
    system_prompt=SUPERVISOR_PROMPT,
    checkpointer=checkpointer,
    subagents=[research_subagent, calendar_specialist_subagent],
)

print("✅ Deep agent with subagents created!")
print("   - research-specialist")
print("   - calendar-specialist")
print("   - general-purpose (built-in)")

✅ Deep agent with subagents created!
   - research-specialist
   - calendar-specialist
   - general-purpose (built-in)


### Test Subagent Delegation

In [6]:
config = {"configurable": {"thread_id": "subagent-demo-1"}}

# Complex request that benefits from subagents
result = agent_11.invoke({
    "messages": [{
        "role": "user",
        "content": "Find upcoming concerts in Seoul this month and schedule the classical concert in my calendar"
    }]
}, config=config)

print("Agent Response:")
print(result['messages'][-1].content)

# Check if files were created by subagents
if "files" in result:
    print("\n📁 Files created by subagents:")
    for path in result["files"].keys():
        print(f"   {path}")

Agent Response:
I couldn't find specific upcoming classical concerts in Seoul for January 2024. It seems the search returned information about events in late 2025 instead. 

For classical music, I recommend checking venues like the Seoul Arts Center or the Seoul Philharmonic Orchestra's official website for updated listings. If you have any specific concert details, please share them, and I can help schedule it in your calendar.

📁 Files created by subagents:


---

# Step 12: Deep Agents with Custom Middleware

## Key Concepts

Deep agents come with built-in middleware:
- **TodoListMiddleware** - Planning with `write_todos`
- **FilesystemMiddleware** - File operations
- **SubAgentMiddleware** - Subagent delegation

You can add custom middleware on top:
- **HumanInTheLoopMiddleware** (via `interrupt_on` parameter)
- **SecurityGuardrailMiddleware** (custom)
- Any other custom middleware

## Middleware Execution Order

```
Custom Middleware (before_agent)
  ↓
Built-in TodoListMiddleware
  ↓
Built-in FilesystemMiddleware
  ↓
Built-in SubAgentMiddleware
  ↓
Tool Execution
  ↓
Built-in Middleware (after_tools)
  ↓
Custom Middleware (after_tools)
```

In [ ]:
# Reset calendar
_calendar_events.clear()

@tool
def delete_calendar_event(date: str, time: str) -> str:
    """Delete a calendar event. This is a sensitive operation that requires confirmation."""
    for event in _calendar_events:
        if event["date"] == date and event["time"] == time:
            title = event["title"]
            _calendar_events.remove(event)
            return f"Successfully deleted event '{title}' on {date} at {time}"
    return f"No event found for {date} at {time}"

# System prompt
SYSTEM_PROMPT_12 = """You are a calendar assistant with planning and file system capabilities.

Available operations:
- read_calendar: Read calendar events
- write_calendar: Create new events
- delete_calendar_event: Delete events (requires confirmation)

Built-in deep agent tools:
- write_todos: Plan multi-step tasks
- File system: ls, read_file, write_file, edit_file
- task: Delegate to subagents

Use write_todos for multi-step operations and file system to save notes or drafts."""

# Create deep agent with interrupt_on for tool confirmation
agent_12 = create_deep_agent(
    model=model,
    tools=[read_calendar, write_calendar, delete_calendar_event],
    system_prompt=SYSTEM_PROMPT_12,
    checkpointer=checkpointer,
    # Add human-in-the-loop for sensitive operations
    interrupt_on={
        "delete_calendar_event": True,  # Require confirmation for deletes
    },
)

print("✅ Deep agent with custom middleware created!")
print("   - Built-in: TodoListMiddleware, FilesystemMiddleware, SubAgentMiddleware")
print("   - Custom: HumanInTheLoopMiddleware (via interrupt_on)")

### Test Normal Operation

In [ ]:
# Pre-populate calendar
_calendar_events.append({
    "title": "Team Standup",
    "date": "2025-12-20",
    "time": "10:00",
    "location": "Zoom"
})

config = {"configurable": {"thread_id": "middleware-demo-1"}}

result = agent_12.invoke({
    "messages": [{"role": "user", "content": "What's on my calendar?"}]
}, config=config)

print("Agent Response:")
print(result['messages'][-1].content)

### Test Human-in-the-Loop (Tool Confirmation)

In [ ]:
from langgraph.types import Command

# Request to delete an event
result = agent_12.invoke({
    "messages": [{"role": "user", "content": "Delete the meeting on 2025-12-20 at 10:00"}]
}, config=config)

# Check for interrupt
if "__interrupt__" in result:
    print("🔔 Agent interrupted! Waiting for confirmation...")
    interrupt_info = result["__interrupt__"][0]
    print(f"   Tool: {interrupt_info.value['action_requests'][0]['name']}")
    print(f"   Args: {interrupt_info.value['action_requests'][0]['args']}")
    print("\n   Simulating user approval...")
    
    # Resume with approval
    result = agent_12.invoke(
        Command(resume={"decisions": [{"type": "approve"}]}),
        config=config
    )
    
    print("\nAgent Response after approval:")
    print(result['messages'][-1].content)
else:
    print("Agent Response:")
    print(result['messages'][-1].content)

---

# Summary: Deep Agents vs Regular Agents

## Regular Agents (`create_agent`)
- ✅ Simple, single-purpose agents
- ✅ Fine-grained control over execution
- ✅ Lightweight applications
- ✅ Direct tool-calling patterns
- ⚠️ Manually add all middleware
- ⚠️ Manual context management
- ⚠️ No built-in planning or file system

## Deep Agents (`create_deep_agent`)
- ✅ Complex, multi-step tasks
- ✅ Built-in planning with `write_todos`
- ✅ Built-in file system for context management
- ✅ Built-in subagent delegation
- ✅ Automatic context eviction and summarization
- ⚠️ More overhead for simple tasks
- ⚠️ Less control over execution details

## When to Use Each

**Use Regular Agents:**
- Simple, focused agents
- Learning agent fundamentals
- Lightweight applications
- Direct tool calling is sufficient

**Use Deep Agents:**
- Complex, multi-step tasks
- Large context management
- Tasks requiring subagent delegation
- Long-running research or analysis
- Production agents inspired by Claude Code, Deep Research

---

## Further Reading

- **Deep Agents Documentation**: `docs/DEEP_AGENTS.md`
- **Workshop Progression**: `docs/deep_agents_progression.md`
- **LangChain Docs**: [docs.langchain.com](https://docs.langchain.com)
- **Deep Agents Package**: [pypi.org/project/deepagents/](https://pypi.org/project/deepagents/)